In [2]:
import pandas as pd
import urllib.request
import numpy as np
import requests
import unicodedata
from PIL import Image

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

## Essential functions

In [3]:
def sanitizar_nombre(name):
    return strip_accents((name.strip().replace(" ", "_").replace("(","").replace(")","")).lower())

# RECIBE FILA DEL DATAFRAME
# precondición: el campo imagen no es NaN
def obtener_url_img(y):
    url = y[7].split('":"')[1]
    url2 = url.split(",")[0][:-1]
    return url2

# RECIBE FILA DEL DATAFRAME
def obtenerlink(y):
    #print(y)
    url = y[5].split('":"')[1]
    url2 = url2 = url.split(",")[0][:-1]
    return url2

def quote(str):
    return '"'+str+'"'


# EMPRESAS

In [4]:
# Read Timeglider csv
df = pd.read_csv("./linea_tiempo.csv",index_col=False)

In [5]:
print(df.columns)
df.head()

Index(['startdate', ' enddate', ' title', ' description', ' importance',
       ' link', ' icon', ' image', ' date_display', ' low threshold',
       ' high threshold', ' y_position'],
      dtype='object')


,startdate,enddate,title,description,importance,link,icon,image,date_display,low threshold,high threshold,y_position
0,1830-12-31 18:14:00,1830-12-31 18:14:00,Ledesma,<p></p>,50,"[{""url"":""https://drive.google.com/open?id=199_...",shapes/triangle_gray.png,"{""src"":""https://upload.wikimedia.org/wikipedia...",da,1,100,0
1,1857-08-27 14:08:00,1857-08-27 14:08:00,Ferrocarril del Oeste,<p></p>,50,"[{""url"":""https://drive.google.com/open?id=0B8E...",shapes/triangle_gray.png,NaN,da,1,100,0
2,1865-07-02 00:28:00,1865-07-02 00:28:00,Semino,<p></p>,50,"[{""url"":""https://drive.google.com/open?id=1gSn...",shapes/triangle_gray.png,"{""src"":""https://s3.amazonaws.com/timeglider_us...",da,1,100,0
3,1876-07-01 22:39:00,1876-07-01 22:39:00,Laboratorios Casasco,<p></p>,50,"[{""url"":""https://drive.google.com/open?id=1YlL...",shapes/triangle_gray.png,"{""src"":""https://www.colfarma.info/colfarma/wp-...",da,1,100,0
4,1877-04-10 00:50:00,1877-04-10 00:50:00,CGFSA,<p></p>,50,"[{""url"":""https://drive.google.com/open?id=1aLF...",shapes/triangle_gray.png,"{""src"":""https://static.wixstatic.com/media/89b...",da,1,100,0


## Manually filtering some data and fixing errors

In [6]:
print(df[' title'].unique())
alpargatas=["Fábrica Argentina de Alpargatas","Alpargatas"] #multiple, deberían ser en 1883 todos # pasa a ser Alpargatas
ferrum=["Ferrum"]
siam=["SIAM"]
pirelli=["Pirelli"]
shell = ["Shell"]
ypf = ["YPF"]
torre = ["Frigorífico Lisandro de la Torre"]
gm = ["General Motors"]
felfort = ["Felfort"]
sere=["La Serenísima"]
entel = ["Entel"]
havanna = ["Havanna"]
aa = ["Aerolíneas Argentinas"]
arcor = ["Arcor"]
pv = ["Central Hidroeléctrica Pueblo Viejo"]
renault = ["Renault Argentina"]
zanon = ["Cerámica Fasinpát (ex Zanón)", "Zanón"] #pasa a ser Zanón
casasco = ["Laboratorios Casasco"]
milkaut = ["Milkaut"]
banco_macro = ["Banco Macro"]
multiples = ["Alpargatas","Ferrum","SIAM","Pirelli","Shell","YPF","Frigorífico Lisandro de la Torre","General Motors","Felfort","La Serenísima","Entel","Havanna","Aerolíneas Argentinas","Arcor","Central Hidroeléctrica Pueblo Viejo","Renault Argentina","Zanón","Laboratorios Casasco","Milkaut","Banco Macro"]

['Ledesma' 'Ferrocarril del Oeste' 'Semino' 'Laboratorios Casasco' 'CGFSA'
 'Aguila Saint' 'Hesperdina' 'Rigolleau'
 'Fábrica Argentina de Alpargatas (2)'
 'Fábrica Argentina de Alpargatas (1)' 'Frigorífico Anglo' 'Bunge y Born'
 'Alpargatas (1)' 'Alpargatas (2)' 'Quilmes' 'La Matrona' 'Grimoldi'
 'Cerro Negro' 'British American Tabacco' 'Pirelli (2)' 'Pirelli (1)'
 'La Martona' 'Molinos Río de la Plata S.A.' 'La Cantábrica'
 'Molinos Río de la Plata' 'Banco Galicia' 'Swift' 'Frávega' 'Ferrum (2)'
 'Bayer Argentina S.A.' 'Ferrum (1)' 'SIAM (2)' 'SIAM (1)' 'Terrabusi'
 'Cerveza Schneider' 'Obras Sanitarias de la Nación' 'BGH' 'FORD'
 'Shell (2)' 'Shell (1)' 'Firestone' 'Goodyear' 'Kodak' 'Longvie' 'FIAT'
 'Roemmers' 'YPF (3)' 'YPF (2)' 'YPF (1)'
 'Frigorífico Lisandro de la Torre (1)'
 'Frigorífico Lisandro de la Torre (2)' 'Estancias Las Marías, Taragüi'
 'General Motors (1)' 'General Motors (2)' 'Felfort (2)' 'Felfort (1)'
 'Loma Negra' 'Milkaut' 'Colgate - Palmolive' 'La Serenísima (

In [12]:
lst = ['Ledesma','Ferrocarril del Oeste','Semino','Laboratorios Casasco','CGFSA','Aguila Saint', 'Hesperdina', 'Rigolleau','Fábrica Argentina de Alpargatas', 'Frigorífico Anglo', 'Bunge y Born','Alpargatas', 'Quilmes', 'La Matrona', 'Grimoldi','Cerro Negro', 'British American Tabacco', 'Pirelli','La Martona', 'Molinos Río de la Plata S.A.', 'La Cantábrica','Molinos Río de la Plata', 'Banco Galicia', 'Swift', 'Frávega','Bayer Argentina S.A.', 'Ferrum', 'SIAM', 'Terrabusi','Cerveza Schneider', 'Obras Sanitarias de la Nación', 'BGH', 'FORD', 'Shell', 'Firestone', 'Goodyear', 'Kodak', 'Longvie', 'FIAT','Roemmers', 'YPF','Frigorífico Lisandro de la Torre', 'Estancias Las Marías, Taragüi','General Motors', 'Felfort','Loma Negra', 'Milkaut', 'Colgate - Palmolive', 'La Serenísima', 'Johnson &amp; Johnson','Molina Cañuelas', 'PHILCO', 'Bagó', 'Phillips', 'SanCor', 'Fernet Branca','Dirección General de Fabricaciones Militares', 'Acindar', 'Coca-Cola','Altos Hornos Zapla', 'Guaymallén', 'Techint', 'Gas del Estado','Entel', 'Havanna','Aerolíneas Argentinas', 'Tenaris', 'Arcor', 'Televisión Pública','Embalse de Río Hondo', 'Astillero Río Santiago','Industrias Kaiser Argentina S.A.', 'Correo OCA', 'Flecha Bus', 'Oslé','Peugeot', 'ELMA', 'Canal 13', 'Telefe', 'LOréal', 'Chocón-Cerros colorados','Central Hidroeléctrica Pueblo Viejo', 'Don Satur','Banco Santander Río', 'Freddo', 'Corven', 'ALUAR', 'Correo Argentino','Sheraton', 'Represa el Chocón', 'Karavell', 'Represa Yacyretá','Central Nuclear Atucha', 'Pumper Nic','Renault Argentina', 'Banco Macro', 'Cerámica Fasinpát (ex Zanón)', 'Día', 'Cablevisión', 'Banco Comafi','Central Hidroeléctrica Río Grande', 'Arcos Dorados S.A.', 'Coto', 'Movicom','Telecentro', 'Edenor', 'UNSAM', 'Motomel', 'Pepsico Argentina', 'FARMACITY','Ampliación Av. General Paz', 'Despegar', 'Mercado Libre', 'Hilton', 'Grido','Crocs', 'CTI Móvil - Claro Argentina','Paseo Bajo nivel de Villa Ballester'
]
lst.sort()
print(lst)

['ALUAR', 'Acindar', 'Aerolíneas Argentinas', 'Aguila Saint', 'Alpargatas', 'Altos Hornos Zapla', 'Ampliación Av. General Paz', 'Arcor', 'Arcos Dorados S.A.', 'Astillero Río Santiago', 'BGH', 'Bagó', 'Banco Comafi', 'Banco Galicia', 'Banco Macro', 'Banco Santander Río', 'Bayer Argentina S.A.', 'British American Tabacco', 'Bunge y Born', 'CGFSA', 'CTI Móvil - Claro Argentina', 'Cablevisión', 'Canal 13', 'Central Hidroeléctrica Pueblo Viejo', 'Central Hidroeléctrica Río Grande', 'Central Nuclear Atucha', 'Cerro Negro', 'Cerveza Schneider', 'Cerámica Fasinpát (ex Zanón)', 'Chocón-Cerros colorados', 'Coca-Cola', 'Colgate - Palmolive', 'Correo Argentino', 'Correo OCA', 'Corven', 'Coto', 'Crocs', 'Despegar', 'Dirección General de Fabricaciones Militares', 'Don Satur', 'Día', 'ELMA', 'Edenor', 'Embalse de Río Hondo', 'Entel', 'Estancias Las Marías, Taragüi', 'FARMACITY', 'FIAT', 'FORD', 'Felfort', 'Fernet Branca', 'Ferrocarril del Oeste', 'Ferrum', 'Firestone', 'Flecha Bus', 'Freddo', 'Frigor

In [26]:
# Sometimes this cell has to be ran twice
filtro_alpargatas = (df[' title'].str.contains(alpargatas[0])) | (df[' title'].str.contains(alpargatas[1]))
#df.loc[filtro_alpargatas,' title'] = "Alpargatas"
df[filtro_alpargatas].head()
for i in range(df[filtro_alpargatas].count()[0]):
    df.loc[filtro_alpargatas,' title'].iloc[i] = "Alpargatas ("+str(i)+")"
#df.loc[9,' title'] = "Alpargatas (2)"
#df.loc[12,' title'] = "Alpargatas (3)"
#df.loc[13,' title'] = "Alpargatas (4)"

filtro_zanon = (df[' title'].str.contains(zanon[0])) | (df[' title'].str.contains(zanon[1]))
df[filtro_zanon].head()
for i in range(df[filtro_zanon].count()[0]):
    df.loc[filtro_zanon,' title'].iloc[i] = "Zanón ("+str(i)+")"
#df.loc[118,' title'] = "Zanón (1)"
#df.loc[199,' title'] = "Zanón (2)"

## Create events based on timeglider csv (migration)

In [28]:
# Buscamos crear algo de esta pinta:
'''
var items = new vis.DataSet([
{id: 1, content: 'item 1', start: '2014-04-20'},
{id: 2, content: 'item 2', start: '2014-04-14'},
{id: 3, content: 'item 3', start: '2014-04-18'},
{id: 4, content: 'item 4', start: '2014-04-16', end: '2014-04-19'},
{id: 5, content:  createItem("item 5",'./img/swift.jpg',["https://makobot-prod.github.io/timelines/","blabla"])  , start: '2014-04-25'},
{id: 6, content: 'item 6', start: '2014-04-27', type: 'point'}
]);
'''
df[' image'] = df[' image'].fillna(0)
### PARECERÍA ESTAR ANDANDO
def crear_evento(x):
    idn = sanitizar_nombre(x[2])
    #print(idn)
    #reemplazar NaNs en image por 0!!
    path_img = quote("0")
    width = 0
    start = quote(x[0][:-9])
    name_empresa = x[2]
    
    if(multiples.count(x[2][:-4])==0):
        links = "["+ quote(obtenerlink(x)) + "]" #crear esta func
    else:
        #tengo multiples trabajos de esta empresa
        #en algun lado me tengo que guardar los múltiples
        name_empresa = x[2][:-4]
        idn = idn[:-2]
        filtro = df[' title'].str.contains(x[2][:-4])
        df_links = df[filtro]
        links = "["
        df_links = df_links.apply(obtenerlink,axis=1)
        for item in df_links.iteritems():
            url = item[1]
            links += quote(url) + ","
        links = links[:-1] + "]" #saco la , al pedo y agrego el corchete final
    if(x[7]!= 0):
        path_img = quote('./img/'+idn+'.png')
        im = Image.open('./timelines/img/'+idn+'.png')
        width, height = im.size
        width = int(width*48/height)
    
    content  = "createItem("+ quote(name_empresa) +","+ path_img +","+ str(width) +","+ links + ")"
    
    return "{id: " + quote(idn) + ", content: " + content + ", start: " + start + "}"
            
#crear_evento(df.loc[118])
#df.loc[118]
filtro = df[' title'].str.contains("Banco Macro")
df[filtro]

,startdate,enddate,title,description,importance,link,icon,image,date_display,low threshold,high threshold,y_position
130,1978-09-04 11:53:00,1978-09-04 11:53:00,Banco Macro (1),<p></p>,50,"[{""url"":""https://drive.google.com/open?id=10Zg...",shapes/triangle_gray.png,"{""src"":""https://s3.amazonaws.com/timeglider_us...",da,1,100,0
131,1978-09-04 23:57:00,1978-09-04 23:57:00,Banco Macro (2),<p></p>,50,"[{""url"":""https://drive.google.com/open?id=165G...",shapes/triangle_gray.png,0,da,1,100,0


#### Apply to dataframe

In [29]:
start_line = "var items = new vis.DataSet(["
end_line = "]);"

df_sin_repes = df.copy()
filtro_repes = (df[' title'].str.contains("\(")) & (~(df[' title'].str.contains("\(1\)")))
df_sin_repes = df_sin_repes[~filtro_repes]

f = open("./timelines/empresas/trabajos_appendable.js", "a")
#f.write(start_line)
f.write("\n")

print("Procesando base de datos...")
cosas = df_sin_repes.apply(crear_evento,axis=1)
print("Guardando en archivo...")
for linea in cosas[:-1]:
    #lleno con todos los eventos hasta el anteultimo
    f.write(linea + ",\n")

#La ULTIMA fila debe ir SIN COMA en la base finalizada, y CON COMA en la appendable!
f.close()
print("Diferenciando versiones appendable y finalizada")
!cp "./timelines/empresas/trabajos_appendable.js" "./timelines/trabajos.js"

f2 = open("./timelines/empresas/trabajos.js","a")
#aca va sin coma y con linea finalizadora
f2.write(cosas.iloc[-1]+"\n")
f2.write(end_line)
f2.close()
print("Version finalizada creada")

#La version appendable va con coma y sin linea finalizadora
f = open("./timelines/empresas/trabajos_appendable.js", "a")
f.write(cosas.iloc[-1]+",")
f.close()
print("Versión appendable creada")
print("Finalizado!")

Procesando base de datos...
Guardando en archivo...
Diferenciando versiones appendable y finalizada
Version finalizada creada
Versión appendable creada
Finalizado!


## Image resizer (OVERWRITES IMAGES)

#### OBSOLETE: images are not that heavy and look better without resizing

In [35]:
#!convert "./timelines/empresas/img/*.png" -filter Lanczos -resize x48 -sharpen 0x.5 -set filename:base "%[basename]" "./timelines/empresas/img/%[filename:base].png"

Decoded /tmp/magick-3689dVLyjnH2RKVf. Dimensions: 560 x 210  (with alpha). Format: lossless. Now saving...
Saved file /tmp/magick-3689r4TsGY9Aa3Al
convert-im6.q16: improper image header `./timelines/img/coto.png' @ error/png.c/ReadPNGImage/3954.
convert-im6.q16: improper image header `./timelines/img/sheraton.png' @ error/png.c/ReadPNGImage/3954.


In [ ]:
#Delete _1 from filename for images that end in _1
#!for file in `find . -name '*_1.png'` ; do mv "$file" "${file%_1.png}.png"; done

## Timeline event images downloader

In [ ]:
#urllib.request.urlretrieve("http://www.digimouth.com/news/media/2011/09/google-logo.jpg", "local-filename.jpg")
# Quiero hacer una función que, dada una fila, me baje la imagen
df_img=df.copy()
df_img.dropna(subset=[' image'], inplace=True) 
falladas=[]
assert(False)
#path = "./img_originales/"
def image_download(x):
    empresa=sanitizar_nombre(x[2])
    # donde x es una serie de Pandas
    url = x[7].split('":"')[1]
    url2 = url.split(",")[0][:-1]
    print(url2)
    try:
        #response = requests.get(url2)
        print("Downloading img for",empresa)
        urllib.request.urlretrieve(url2, path + empresa.lower() + ".png")
    except Exception as exc:
        print('[!!!] {err}'.format(err=exc),"retrieving img for", empresa ,"aborted")
        falladas.append(empresa)

for index, row in df_img.iterrows():
    print(index)
    image_download(df_img.loc[index])


In [99]:
print(falladas)

['cgfsa', 'correo_oca', 'don_satur', 'sheraton', 'arcos_dorados_s.a.', 'telecentro']


In [18]:
if(not np.isnan(df.loc[1][7].isnull()):
    print("Not a nan")
    print(df.loc[1][7])
    urllib.request.urlretrieve(df.loc[1][7], path+df.loc[1][2].lower()+".jpg")

# HISTORIC EVENTS TIMELINES

In [108]:
unit_name="4unidad"
df = pd.read_csv("./"+unit_name+"/"+unit_name+".csv",index_col=False)
df.columns = df.columns.str.replace(' ', '')
logo_info = "www.cooperacion2005.es"
logo_tour = "guejarsierra.org"

In [109]:
df.head()

,startdate,enddate,title,description,importance,link,icon,image,date_display,lowthreshold,highthreshold,y_position
0,1929-10-24 20:56:00,1929-10-24 20:56:00,24-10-1929: crack de la Bolsa de New York,<p></p>,75,"[{""url"":""https://drive.google.com/file/d/1FRG8...",shapes/triangle_gray.png,"{""src"":""http://www.cooperacion2005.es/wp-conte...",da,1,100,0
1,1930-01-01 18:22:00,1930-01-01 18:22:00,Década de 1930: contexto histórico internacional,<p></p>,60,"[{""url"":""https://drive.google.com/file/d/1dw-R...",shapes/triangle_gray.png,"{""src"":""http://www.cooperacion2005.es/wp-conte...",da,1,100,0
2,1930-01-18 20:56:00,1930-01-18 20:56:00,18-01-1930: intento frustrado de firmar el con...,<p></p>,60,NaN,shapes/triangle_gray.png,NaN,da,1,100,0
3,1930-07-01 20:56:00,1930-07-01 20:56:00,1930: Gran Depresión del 30,<p></p>,75,NaN,shapes/triangle_gray.png,NaN,ye,1,100,0
4,1930-07-01 20:56:00,1930-07-01 20:56:00,1930 al 1936: &#34;nacionalismo oligárquico&#3...,<p></p>,60,NaN,shapes/triangle_gray.png,NaN,ye,1,100,0


In [110]:
# Identify type of timeline event (general information (logo_info) vs tourist information (logo_tour))
def find_type(event):
    if(not pd.isnull(event[7])):
        #not a NaN
        if(event[7][15:37]==logo_info):
            return "informacion"
        elif(event[7][15:31]==logo_tour):
            return "turismo"
    else:
        return 0
    
print(find_type(df.iloc[7]))

informacion


## Migrate events from Timeglider

In [111]:
# First, we must identify possible repeated events

def sanitizar_nombre(name):
    return strip_accents((name.strip().replace(" ", "_").replace("(","").replace(")","")).lower()).replace('&#34;','')

def get_name(x):
    sanitized = sanitizar_nombre(x[2])
    return sanitized[:35]

names = df.apply(get_name,axis=1)
seen = set()
check_repeats = []
for x in names:
    if x not in seen:
        seen.add(x)
    else:
        check_repeats.append(x)
handled_repeats=[]
print(check_repeats)

['07-09-1930:_ministro_de_economia_e.', '2-5-1935:_fundacion_del_iram_en_arg', '27-11-1943:_peron_es_designado_secr', '06_y_09-08-1945:_bombardeos_atomico']


In [114]:
def crear_evento(x):
    idn = get_name(x)
    #print(idn)
    #reemplazar NaNs en image por 0!!
    path_img = quote("0")
    width = 0
    start = quote(x[0][:-9])
    end = "0"
    if(x[0][:-9] != x[1][:-9]):
        end = quote(x[1][:-9])
    
    name_event = x[2].replace("&#34;", '\\"')
    count_links = 0
    
    
    if(check_repeats.count(idn)==0):
        #Event is not repeated
        if(not pd.isnull(x[5])):
            count_links = 1
            links = "["+ quote(obtenerlink(x)) + "]" 
    else:
        if(handled_repeats.count(idn)!=0):
            raise Exception("Event already created") 
        handled_repeats.append(idn) #we only want one event with all the data
        if(not pd.isnull(x[5])):
            count_links = 2 #more than 1 link = more than 1 work on same event
            filtro = df['title'].str.contains(x[2][:21])
            df_links = df[filtro]
            links = "["
            df_links = df_links.apply(obtenerlink,axis=1)
            for item in df_links.iteritems():
                url = item[1]
                links += quote(url) + ","
            links = links[:-1] + "]" #saco la , al pedo y agrego el corchete final
    
    if(pd.isnull(x[5])):
        links = "["+"]"
    
    img = find_type(x)
    content  = quote(name_event) 
    
    if(img!= 0):
        #if there is an image, include image
        path_img = quote('../img/'+img+'.svg')
        content += ", image:"+ path_img 
        
    if(count_links>1):
        #if there is multiple links, pass list of links
        content += ", links:" + links
    elif(count_links==1):
        #if there is only one link, pass single link
        content += ", link:" + links[1:][:-1]
    
    res = "{id: " + quote(idn) + ", content: " + content + ", start: " + start 
    if(end!="0"):
        res += ", end: " + end
        
    res += "}"
    return res

def try_create(x):
    try:
        return crear_evento(x)
        # If exception is raised, no return
    except Exception as exc:
        print('[!!!] Evento repetido:',x[2])

print("event:",crear_evento(df.iloc[5]))
print(handled_repeats)

event: {id: "06-09-1930_golpe_de_estado_de_uribu", content: "06-09-1930 Golpe de Estado de Uriburu (bis)", image:"../img/informacion.svg", link:"https://drive.google.com/file/d/1gCIWrsk8TO1PMrxQ72PxFs8Gbx7flpPk/view?usp=sharing", start: "1930-09-06"}
['07-09-1930:_ministro_de_economia_e.', '2-5-1935:_fundacion_del_iram_en_arg', '27-11-1943:_peron_es_designado_secr', '06_y_09-08-1945:_bombardeos_atomico']


### Apply to dataframe

In [117]:
start_line = "var items = new vis.DataSet(["
end_line = "]);"

handled_repeats = []

f = open("./"+unit_name+"/"+unit_name+"_appendable.js", "w")
f.write(start_line)
f.write("\n")

print("Procesando base de datos...")
df_copy = df.copy()
cosas = df_copy.apply(try_create,axis=1)
cosas = cosas.dropna()
print("Guardando en archivo...")
for linea in cosas[:-1]:
    #lleno con todos los eventos hasta el anteultimo
    f.write(linea + ",\n")

#La ULTIMA fila debe ir SIN COMA en la base finalizada, y CON COMA en la appendable!
f.close()
print("Diferenciando versiones appendable y finalizada")
!cp "./{unit_name}/{unit_name}_appendable.js" "./{unit_name}/{unit_name}.js"

f2 = open("./"+unit_name+"/"+unit_name+".js","a")
#aca va sin coma y con linea finalizadora
f2.write(cosas.iloc[-1]+"\n")
f2.write(end_line)
f2.close()
print("Version finalizada creada")

#La version appendable va con coma y sin linea finalizadora
f = open("./"+unit_name+"/"+unit_name+"_appendable.js", "a")
f.write(cosas.iloc[-1]+",")
f.close()
print("Versión appendable creada")
print("Finalizado!")

Procesando base de datos...
[!!!] Evento repetido: 07-09-1930: Ministro de Economía E. Pérez (hasta 16-04-1931)
[!!!] Evento repetido: 2-5-1935: fundación del IRAM en Argentina
[!!!] Evento repetido: 27-11-1943: Perón es designado Secretario de Trabajo y Previsión
[!!!] Evento repetido: 06 y 09-08-1945: bombardeos atómicos de Hiroshima y Nagasaki, respectivamente
[!!!] Evento repetido: 06 y 09-08-1945: bombardeos atómicos de Hiroshima y Nagasaki, respectivamente
Guardando en archivo...
Diferenciando versiones appendable y finalizada
Version finalizada creada
Versión appendable creada
Finalizado!


## New events from google forms

In [14]:
df = pd.read_csv("./empresas/new_groups.csv",index_col=False)
df['img'] = df['img'].fillna(0)
df.head(10)

,name,date,img,work,present
0,Llao Llao Hotel & Resort Golf-Spa,27/9/2020,./img/llao_llao_hotel_&_resort_golf-spa.png,https://drive.google.com/file/d/1C7i8-12lf2cG5...,No
1,La Paulina,8/6/1921,./img/la_paulina.png,https://drive.google.com/file/d/1PL5rrj5FxHQfM...,No
2,Marolio,1/1/1950,./img/marolio.png,https://drive.google.com/file/d/1PI-JZWnAGplNZ...,No
3,Cafe Tortoni,1/6/1858,./img/cafe_tortoni.png,https://drive.google.com/file/d/1GdkkGuiUMhfZr...,No
4,Pizzini,1/1/1955,0,https://drive.google.com/file/d/10zAfBOqmS4s0B...,No
5,La Virginia,29/11/1915,./img/la_virginia.png,https://drive.google.com/file/d/1p96yyX2KoC-Ln...,No
6,La Cachuera,8/6/1901,./img/la_cachuera.png,https://drive.google.com/file/d/1-wk9heUUpr3T4...,No
7,SEGBA (Servicios Eléctricos del Gran Buenos Ai...,1/10/1958,./img/segba_servicios_electricos_del_gran_buen...,https://drive.google.com/file/d/19idXKDm4ZOf5Q...,No
8,Volkswagen,28/5/1937,./img/volkswagen.png,https://drive.google.com/file/d/1yAGqeqnvtRvd7...,No
9,Hotel Alvear Palace,1/1/1932,./img/hotel_alvear_palace.png,https://drive.google.com/file/d/1NHDqKt05jC-D0...,No


#### Nuevas empresas (no aparecieron antes en la linea)

In [18]:
df = df[df['present']=='No']
#Necesito ver si hay repetidos dentro de las entregas nuevas
#First, we must identify possible repeated events among new entries

def get_name(x):
    return sanitizar_nombre(x[0])

#Generamos lista de elementos repetidos entre los nuevos, contra la que chequearemos para no agregarlos mas de una vez
names = df.apply(get_name,axis=1)
seen = set()
check_repeats = []
for x in names:
    if x not in seen:
        seen.add(x)
    else:
        check_repeats.append(x)
handled_repeats=[]
print(check_repeats)

[]


In [30]:
def parseDate(date):
    numbers = date.split("/")
    return numbers[2]+"-"+numbers[1]+"-"+numbers[0]
  
def crear_evento(x):
    idn = sanitizar_nombre(x[0])
    path_img = quote("0")
    width = 0
    start = quote(parseDate(x[1]))
    name_empresa = x[0]
    count_links = 0
    
    if(not pd.isnull(x[3])):
        if(check_repeats.count(idn)==0):
            #Event is not repeated
            count_links = 1
            links = "["+ quote(x[3]) + "]" 
        else:
            print(handled_repeats)
            if(handled_repeats.count(idn)!=0):
                print('[!!!] Evento repetido:',x[0])
                raise Exception("Event already created") #Cancel processing if event was already created for this enterprise
            handled_repeats.append(idn) #we only want one event with all the data
            count_links = 2 #more than 1 link = more than 1 work on same event
            filtro = df['name'].str.contains(x[0])
            df_links = df[filtro]
            links = "["
            for item in df_links.iteritems():
                print(item)
                raise Exception("Testear que esto funcione") 
                url = item[2]
                links += quote(url) + ","
            links = links[:-1] + "]" #saco la , al pedo y agrego el corchete final
    else:
        links = "["+"]"
    
    if(x[2]!= 0):
        path_img = quote(x[2])
        jupyter_path = "./empresas/img/"+idn+".png"
        im = Image.open(jupyter_path)
        width, height = im.size
        width = int(width*48/height)
    
    content  = "createItem("+ quote(name_empresa) +","+ path_img +","+ str(width) +","+ links + ")"
    
    res = "{id: " + quote(idn) + ", content: " + content + ", start: " + start 
        
    res += "}"
    return res

def try_create(x):
    try:
        return crear_evento(x)
        # If exception is raised, no return
    except Exception as exc:
        print('ERROR '+ x[0])

print("event:",crear_evento(df.iloc[4]))
print(handled_repeats)

event: {id: "pizzini", content: createItem("Pizzini","0",0,["https://drive.google.com/file/d/10zAfBOqmS4s0BoeDtpvH2u90RHUBV-XZ/view?usp=drivesdk"]), start: "1955-1-1"}
[]


In [31]:
start_line = "var items = new vis.DataSet(["
end_line = "]);"

#Necesito vaciar la lsita de repetidos antes de procesar todo el df
handled_repeats = []

!cp ./empresas/trabajos_appendable.js ./empresas/trabajos_appendable_backup.js
!cp ./empresas/trabajos.js ./empresas/trabajos_backup.js
f = open("./empresas/trabajos_appendable.js", "a")
#f.write(start_line)
#f.write("\n")

print("Procesando base de datos...")
df_copy = df.copy()
cosas = df_copy.apply(try_create,axis=1)
cosas = cosas.dropna() #Drop lines that threw exception (repeated enterprise event)
print("Guardando en archivo...")
for linea in cosas[:-1]:
    #Write up to second-to-last event in file
    f.write(linea + ",\n")

#La ULTIMA fila debe ir SIN COMA en la base finalizada, y CON COMA en la appendable!
f.close()
print("Diferenciando versiones appendable y finalizada")
!cp "./empresas/trabajos_appendable.js" "./empresas/trabajos.js"

f2 = open("./empresas/trabajos.js","a")
#aca va sin coma y con linea finalizadora
f2.write(cosas.iloc[-1]+"\n")
f2.write(end_line)
f2.close()
print("Version finalizada creada")

#La version appendable va con coma y sin linea finalizadora
f = open("./empresas/trabajos_appendable.js", "a")
f.write(cosas.iloc[-1]+",")
f.close()
print("Versión appendable creada")
print("Finalizado!")

Procesando base de datos...
Guardando en archivo...
Diferenciando versiones appendable y finalizada
Version finalizada creada
Versión appendable creada
Finalizado!


### Empresas que ya estaban presentes en la linea

In [36]:
# Lo que habría que hacer acá es copiar trabajos_appendable.js y reimprimirla modificando las lineas que corresponden.
df2 = df[df['present']=='Si'].count()[0]
print(df2)

0
